In [8]:
# # both are installed at same time  then error occurs... as pymongo also has bson module of some kind so first install 'pymongo[srv]' and then run import in below cell then install bson and run import in below cell again

# ! pip install 'pymongo[srv]'
# ! pip install bson

In [9]:
# ! pip uninstall 'pymongo[srv]' -y
# ! pip uninstall bson -y

In [10]:
from pymongo import MongoClient
from dotenv import load_dotenv
import pprint # pretty print library is used to make the output look more organized
import datetime
from secrets import *


from bson.objectid import ObjectId

In [11]:
MONGODB_URI = connection_string
client = MongoClient(MONGODB_URI)

# # List all databases in the cluster.
# for db_name in client.list_database_names():
#     print(db_name)



### insert_one

In [12]:

# Get reference to 'bank' database
db = client.bank  # if dont exist, it will be created

# Get reference to 'accounts' collection
accounts_collection = db.accounts # if dont exist, it will be created

new_account = {
    "account_holder": "Linus Torvalds",
    "account_id": "MDB829001337",
    "account_type": "checking",
    "balance": 50352434,
    "last_updated": datetime.datetime.utcnow(),
}

# Write an expression that inserts the 'new_account' document into the 'accounts' collection.
result = accounts_collection.insert_one(new_account)

document_id = result.inserted_id
print(f"_id of inserted document: {document_id}")




_id of inserted document: 63806d2e1006d864394f9726


### insert_many

In [13]:
# Get reference to 'bank' database
db = client.bank

# Get a reference to 'accounts' collection
accounts_collection = db.accounts

new_accounts = [
    {
        "account_id": "MDB011235813",
        "account_holder": "Ada Lovelace",
        "account_type": "checking",
        "balance": 60218,
    },
    {
        "account_id": "MDB829000001",
        "account_holder": "Muhammad ibn Musa al-Khwarizmi",
        "account_type": "savings",
        "balance": 267914296,
    },
]

# Write an expression that inserts the documents in 'new_accounts' into the 'accounts' collection.
result = accounts_collection.insert_many(new_accounts)

document_ids = result.inserted_ids
print("# of documents inserted: " + str(len(document_ids)))
print(f"_ids of inserted documents: {document_ids}")

# of documents inserted: 2
_ids of inserted documents: [ObjectId('63806d2f1006d864394f9727'), ObjectId('63806d2f1006d864394f9728')]


### find_one

In [14]:
# Get reference to 'bank' database
db = client.bank

# Get a reference to the 'accounts' collection
accounts_collection = db.accounts

# Query by ObjectId
document_to_find = {"_id": ObjectId("6380516f1279ecfabb3decc9")}

# Write an expression that retrieves the document matching the query constraint in the 'accounts' collection.
result = accounts_collection.find_one(document_to_find)
pprint.pprint(result)



None


### find for all

In [15]:
# Get reference to 'bank' database
db = client.bank

# Get a reference to the 'accounts' collection
accounts_collection = db.accounts

# Query
documents_to_find = {"balance": {"$gt": 4700}}

# Write an expression that selects the documents matching the query constraint in the 'accounts' collection.
cursor = accounts_collection.find(documents_to_find)  # cursor is a pointer to the result set of the query which we can iterate over

num_docs = 0
for document in cursor:
    num_docs += 1
    pprint.pprint(document)
    print()
print("# of documents found: " + str(num_docs))

{'_id': ObjectId('63804f3e03cc11cd1f77565d'),
 'account_holder': 'Linus Torvalds',
 'account_id': 'MDB829001337',
 'account_type': 'checking',
 'balance': 50352434,
 'last_updated': datetime.datetime(2022, 11, 25, 5, 14, 38, 762000)}

{'_id': ObjectId('63804fcf03cc11cd1f775663'),
 'account_holder': 'Linus Torvalds',
 'account_id': 'MDB829001337',
 'account_type': 'checking',
 'balance': 50352434,
 'last_updated': datetime.datetime(2022, 11, 25, 5, 17, 3, 265000)}

{'_id': ObjectId('6380510f7917cf02af801200'),
 'account_holder': 'Linus Torvalds',
 'account_id': 'MDB829001337',
 'account_type': 'checking',
 'balance': 50352434,
 'last_updated': datetime.datetime(2022, 11, 25, 5, 22, 23, 859000)}

{'_id': ObjectId('6380516f1279ecfabb3decc8'),
 'account_holder': 'Ada Lovelace',
 'account_id': 'MDB011235813',
 'account_type': 'checking',
 'balance': 60218}

{'_id': ObjectId('638052a603cc11cd1f775664'),
 'account_holder': 'Ada Lovelace',
 'account_id': 'MDB011235813',
 'account_type': 'check

### update_one

In [16]:
# Get reference to 'bank' database
db = client.bank

# Get reference to 'accounts' collection
accounts_collection = db.accounts

# Filter
document_to_update = {"_id": ObjectId("6380516f1279ecfabb3decc9")}

# Update
add_to_balance = {"$inc": {"balance": 100}}

# Print original document
pprint.pprint(accounts_collection.find_one(document_to_update))

# Write an expression that adds to the target account balance by the specified amount.
result = accounts_collection.update_one(document_to_update, add_to_balance)
print("Documents updated: " + str(result.modified_count))

# Print updated document
pprint.pprint(accounts_collection.find_one(document_to_update))

None
Documents updated: 0
None


### update_many

In [17]:
# Get reference to 'bank' database
db = client.bank

# Get reference to 'accounts' collection
accounts_collection = db.accounts

# Filter
select_accounts = {"account_type": "savings"}

# Update
set_field = {"$set": {"minimum_balance": 100}}

# Write an expression that adds a 'minimum_balance' field to each savings acccount and sets its value to 100.
result = accounts_collection.update_many(select_accounts, set_field)

print("Documents matched: " + str(result.matched_count))
print("Documents updated: " + str(result.modified_count))
pprint.pprint(accounts_collection.find_one(select_accounts))

Documents matched: 6
Documents updated: 1
{'_id': ObjectId('638052a603cc11cd1f775665'),
 'account_holder': 'Muhammad ibn Musa al-Khwarizmi',
 'account_id': 'MDB829000001',
 'account_type': 'savings',
 'balance': 267914296,
 'minimum_balance': 100}


### delete_one     
delete_one({})  delete the first document

In [18]:
# Get reference to 'bank' database
db = client.bank

# Get a reference to the 'accounts' collection
accounts_collection = db.accounts

# Filter by ObjectId
document_to_delete = {"_id": ObjectId("6380516f1279ecfabb3decc9")}

# Search for document before delete
print("Searching for target document before delete: ")
pprint.pprint(accounts_collection.find_one(document_to_delete))

# Write an expression that deletes the target account.
result = accounts_collection.delete_one(document_to_delete)

# Search for document after delete
print("Searching for target document after delete: ")
pprint.pprint(accounts_collection.find_one(document_to_delete))

print("Documents deleted: " + str(result.deleted_count))


Searching for target document before delete: 
None
Searching for target document after delete: 
None
Documents deleted: 0


### delete_many

In [19]:
# Get reference to 'bank' database
db = client.bank

# Get a reference to the 'accounts' collection
accounts_collection = db.accounts

# Filter for accounts with balance less than $2000
documents_to_delete = {"balance": {"$lt": 2000}}

# Search for sample document before delete
print("Searching for sample target document before delete: ")
pprint.pprint(accounts_collection.find_one(documents_to_delete))

# Write an expression that deletes the target accounts.
result = accounts_collection.delete_many(documents_to_delete)

# Search for sample document after delete
print("Searching for sample target document after delete: ")
pprint.pprint(accounts_collection.find_one(documents_to_delete))

print("Documents deleted: " + str(result.deleted_count))

Searching for sample target document before delete: 
None
Searching for sample target document after delete: 
None
Documents deleted: 0


### mongo db transaction

In [20]:
# Connect to MongoDB cluster with MongoClient
client = MongoClient(MONGODB_URI)

# Step 1: Define the callback that specifies the sequence of operations to perform inside the transactions.
def callback(
    session,
    transfer_id=None,
    account_id_receiver=None,
    account_id_sender=None,
    transfer_amount=None,
):

    # Get reference to 'accounts' collection
    accounts_collection = session.client.bank.accounts

    # Get reference to 'transfers' collection
    transfers_collection = session.client.bank.transfers

    transfer = {
        "transfer_id": transfer_id,
        "to_account": account_id_receiver,
        "from_account": account_id_sender,
        "amount": {"$numberDecimal": transfer_amount},
    }

    # Transaction operations
    # Important: You must pass the session to each operation

    # Update sender account: subtract transfer amount from balance and add transfer ID
    accounts_collection.update_one(
        {"account_id": account_id_sender},
        {
            "$inc": {"balance": -transfer_amount},
            "$push": {"transfers_complete": transfer_id}, # push is used to add to an array
        },
        session=session,
    )

    # Update receiver account: add transfer amount to balance and add transfer ID
    accounts_collection.update_one(
        {"account_id": account_id_receiver},
        {
            "$inc": {"balance": transfer_amount},
            "$push": {"transfers_complete": transfer_id},
        },
        session=session,
    )

    # Add new transfer to 'transfers' collection
    transfers_collection.insert_one(transfer, session=session)

    print("Transaction successful")

    return


def callback_wrapper(s):
    callback(
        s,
        transfer_id="TR218721873",
        account_id_receiver="MDB343652528",
        account_id_sender="MDB574189300",
        transfer_amount=100,
    )


# Step 2: Start a client session
with client.start_session() as session:
    # Step 3: Use with_transaction to start a transaction, execute the callback, and commit (or cancel on error)
    session.with_transaction(callback_wrapper)

Transaction successful


### update and group

In [21]:
# Connect to MongoDB cluster with MongoClient
client = MongoClient(MONGODB_URI)

# Get reference to 'bank' database
db = client.bank

# Get reference to 'accounts' collection
accounts_collection = db.accounts

# Calculate the average balance of checking and savings accounts with balances of less than $1000.

# Select accounts with balances of greater than $1000.
select_by_balance = {"$match": {"balance": {"$gt": 1000}}}

# Separate documents by account type and calculate the average balance for each account type.
separate_by_account_calculate_avg_balance = {
    "$group": {"_id": "$account_type", "avg_balance": {"$avg": "$balance"}}
}

# Create an aggegation pipeline using 'stage_match_balance' and 'stage_group_account_type'.
pipeline = [
    select_by_balance,
    separate_by_account_calculate_avg_balance,
]

# Perform an aggregation on 'pipeline'.
results = accounts_collection.aggregate(pipeline)

print()
print(
    "Average balance of checking and savings accounts with balances of less than $1000:", "\n"
)

for item in results:
    pprint.pprint(item)


Average balance of checking and savings accounts with balances of less than $1000: 

{'_id': 'savings', 'avg_balance': 267914296.0}
{'_id': 'checking', 'avg_balance': 23272010.0}


### sort and projection


In [22]:
# Connect to MongoDB cluster with MongoClient
client = MongoClient(MONGODB_URI)

# Get reference to 'bank' database
db = client.bank

# Get a reference to the 'accounts' collection
accounts_collection = db.accounts

# Return the account type, original balance, and balance converted to Great British Pounds (GBP)
# of all checking accounts with an original balance of greater than $1,500 US dollars, in order from highest original balance to lowest.

# To calculate the balance in GBP, divide the original balance by the conversion rate
conversion_rate_usd_to_gbp = 1.3

# Select checking accounts with balances of more than $1,500.
select_accounts = {"$match": {"account_type": "checking", "balance": {"$gt": 1500}}}

# Organize documents in order from highest balance to lowest.
organize_by_original_balance = {"$sort": {"balance": -1}}

# Return only the account type & balance fields, plus a new field containing balance in Great British Pounds (GBP).
return_specified_fields = {
    "$project": {
        "account_type": 1,
        "balance": 1,
        "gbp_balance": {"$divide": ["$balance", conversion_rate_usd_to_gbp]},
        "_id": 0,
    }
}

# Create an aggegation pipeline containing the four stages created above
pipeline = [
    select_accounts,
    organize_by_original_balance,
    return_specified_fields,
]

# Perform an aggregation on 'pipeline'.
results = accounts_collection.aggregate(pipeline)

print(
    "Account type, original balance and balance in GDP of checking accounts with original balance greater than $1,500,"
    "in order from highest original balance to lowest: ", "\n"
)

for item in results:
    pprint.pprint(item)

Account type, original balance and balance in GDP of checking accounts with original balance greater than $1,500,in order from highest original balance to lowest:  

{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking',
 'balance': 50352434,
 'gbp_balance': 38732641.538461536}
{'account_type': 'checking', 'balance': 60218, 'gbp_balance': 46321.53846153846}
{'account_type': 'checking', 'balance': 60218, 'gbp_balance': 46321.53846153846}
{'account_type': 'checking', 'balance': 60218, 'gbp_balance': 46321.53846153846}
{'account_type': 'checking', 'balance': 60218, 'gbp_balance': 46321.5